In [5]:
import cv2
import numpy as np
cap=cv2.VideoCapture(0)
k=0
blue,green,red=0,0,0
def nothing(x):
    pass
def click_event(event,x,y,flags,param):
    global k,blue,green,red
    cv2.namedWindow("Trackbars")
    cv2.createTrackbar("Lower H", "Trackbars", 0, 179, nothing)
    cv2.createTrackbar("Lower S", "Trackbars", 0, 255, nothing)
    cv2.createTrackbar("Lower V", "Trackbars", 0, 255, nothing)
    cv2.createTrackbar("Upper H", "Trackbars", 179, 179, nothing)
    cv2.createTrackbar("Upper S", "Trackbars", 255, 255, nothing)
    cv2.createTrackbar("Upper V", "Trackbars", 255, 255, nothing)
    if(event==cv2.EVENT_LBUTTONDOWN):
        k=1
        ss=frame
        blue=ss[y,x,0]
        green=ss[y,x,1]
        red=ss[y,x,2]
        while True:
            lh = cv2.getTrackbarPos("Lower H", "Trackbars")
            ls = cv2.getTrackbarPos("Lower S", "Trackbars")
            lv = cv2.getTrackbarPos("Lower V", "Trackbars")
            uh = cv2.getTrackbarPos("Upper H", "Trackbars")
            us = cv2.getTrackbarPos("Upper S", "Trackbars")
            uv = cv2.getTrackbarPos("Upper V", "Trackbars")
            lower=np.array([lh,ls,lv])
            upper=np.array([uh,us,uv])
            hsv=cv2.cvtColor(ss, cv2.COLOR_BGR2HSV)
            mask=cv2.inRange(hsv,lower,upper)
            res=cv2.bitwise_and(ss,ss,mask=mask)
            mask2=cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
            stacked=np.hstack((mask2,res))
            cv2.imshow("Trackbars",cv2.resize(stacked,None,fx=0.4,fy=0.4))
            if cv2.waitKey(1)==ord('s'):
                thearray = [[lh,ls,lv],[uh,us,uv]]
                np.save('penval',thearray)
                break
while(1):
    _,frame=cap.read()
    frame=cv2.flip(frame,1)
    cv2.imshow("Live",frame)
    cv2.setMouseCallback("Live",click_event)
    cv2.waitKey(1) & 0xFF
    if k==1:
        break
cv2.destroyAllWindows()
cap.release()
penval = np.load('penval.npy')
cap2=cv2.VideoCapture(0)
cap2.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
cap2.set(cv2.CAP_PROP_FRAME_HEIGHT,720)
kernel = np.ones((5,5),np.uint8)
board=np.zeros((720,1280,3),np.uint8)
while(1):
    _,frame2=cap2.read()
    frame2=cv2.flip(frame2,1)
    hsv = cv2.cvtColor(frame2,cv2.COLOR_BGR2HSV)
    lower_range=penval[0]
    upper_range=penval[1]
    mask=cv2.inRange(hsv, lower_range, upper_range)
    mask=cv2.erode(mask,kernel,iterations = 1)
    mask=cv2.dilate(mask,kernel,iterations = 2)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > 500:
        c = max(contours, key = cv2.contourArea)
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(frame2,(x+2,y+2),(x+w+2,y+h+2),(51,134,255),2)
        cv2.circle(board,(x,y),7,(int(blue),int(green),int(red)),-1)
    stacked = np.hstack((frame2,board))
    cv2.imshow('Track',stacked)
    if cv2.waitKey(1) & 0xFF==27:
        break
cap2.release()    
cv2.destroyAllWindows()